Column in DF
Description
article_id
From filename or parsed XML metadata
dataset_reference
A candidate URL or DOI from full text or reference section
context_before
~50 words (or characters) before the citation in the sentence or paragraph
context_after
~50 words (or characters) after the citation
full_sentence
(Optional) The whole sentence or paragraph containing the citation
section_title
(Optional) Which section it came from: Abstract, Methods, etc.
ref_type
(Optional, for later) whether it came from <ref> section or inline


In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
from dataset_reference_extractor import DatasetReferenceExtractor
from regex_patterns import patterns

extractor = DatasetReferenceExtractor(regex_patterns=patterns, context_window=40)

df = extractor.run_on_folder(
    pdf_folder="data/train/PDF",
    xml_folder="data/train/XML"
)

df.head()

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: unsupported error: cannot create appearance stream for  annotations

MuPDF error: uns

,row_id,article_id,dataset_id,context,pattern_type,source,start_idx,score
0,0,10.1002_2017jc013030,https://doi.org/10.3354/meps11580,properties of some phytoplankters.Deep Sea Res...,doi_short,pdf,58307,0
1,1,10.1002_2017jc013030,https://doi.org/10.4319/lo.2011.56.3.0955,southern Bay of Biscay .Marine Ecology Progres...,doi_short,pdf,58569,0
2,2,10.1002_2017jc013030,https://doi.org/10.4319/lo.2003.48.2.0843,bio-optical oceanic regimes.Limnology and Ocea...,doi_short,pdf,58828,0
3,3,10.1002_2017jc013030,https://doi.org/10.1016/s0967-0637,J Nolan 10.1016/S0967-0637 01 00025-5 Deep Sea...,doi_xml,xml,57177,0
4,4,10.1002_2017jc013030,https://doi.org/10.4319/lo.1996.41.8.1684,Part I: Oceanographic Research Papers 48 11 24...,doi_short,pdf,59686,0


In [3]:
#df.groupby('article_id').

In [4]:
df['pattern_type'].value_counts()

pattern_type
doi_long                            8265
doi_short                           6851
doi_xml                             2980
PDB ID                               533
EC Numbers                           487
F ID                                 299
Short Alphanumeric ID (PDB-like)     197
UniProt ID                           145
rs ID                                 88
SAMN BioSample                        85
GSE                                   83
SRX/SRR/SRP                           73
PRJNA (BioProject)                    51
KEGG K Numbers                        48
D ID                                  36
EPI_ISL (GISAID)                      35
IPR                                   35
NM_ RefSeq                            33
PF                                    30
CHEMBL                                29
EPI Short (Legacy)                    29
E-GEOD (GEO)                          26
CVCL (Cellosaurus)                    20
EMPIAR                                18
PXD

In [5]:
df['dataset_id'].nunique()

19639

In [6]:
# Number of unique (article_id, dataset_id) pairs
df[['article_id', 'dataset_id']].drop_duplicates().shape[0]

20590

In [7]:
df.shape

(20590, 8)

In [8]:
doi_df = df[df['dataset_id'].str.contains("doi.org", case=False, na=False)]
doi_df.sort_values(by='article_id', inplace=True)
doi_df.head()

,row_id,article_id,dataset_id,context,pattern_type,source,start_idx,score
0,0,10.1002_2017jc013030,https://doi.org/10.3354/meps11580,properties of some phytoplankters.Deep Sea Res...,doi_short,pdf,58307,0
259,259,10.1002_2017jc013030,https://doi.org/10.5194/essd-2017-58.bio-optic...,derived from BGC-Argo float measurements for m...,doi_long,xml,94876,0
258,258,10.1002_2017jc013030,https://doi.org/10.5194/essd-2017-58.bio-optical,derived from BGC-Argo float measurements for m...,doi_short,xml,94876,0
242,242,10.1002_2017jc013030,https://doi.org/10.1364/oe.17.012685journal,Research 116 C6 .https://doi.org/10.1029/2010J...,doi_long,pdf,94063,0
241,241,10.1002_2017jc013030,https://doi.org/10.4319/lo.1989.34.8.1694zhang,processing and mutual interpretation of radiom...,doi_long,pdf,93913,0


In [9]:
doi_df[doi_df['source'] == 'pdf'].shape

(13320, 8)

In [10]:
doi_df[doi_df['dataset_id'] == 'https://doi.org/10.5061/dryad.5q1sb']

,row_id,article_id,dataset_id,context,pattern_type,source,start_idx,score


In [11]:
# Read ground truth labels
import pandas as pd
labels_df = pd.read_csv("data/train_labels.csv")
#labels_df.loc[labels_df['article_id'] == '10.1371_journal.pone.0070749', 'dataset_id'] = 'https://doi.org/10.5061/dryad.5q1sb'

# Ensure we compare only unique combinations
labels_df_cleaned = labels_df[labels_df['dataset_id'] != 'Missing']
label_pairs = set(zip(labels_df_cleaned.article_id, labels_df_cleaned.dataset_id))
extracted_pairs = set(zip(df.article_id, df.dataset_id))

# Missed pairs (False Negatives)
missed = label_pairs - extracted_pairs
print(f"Missed {len(missed)} true (article_id, dataset_id) pairs")

Missed 111 true (article_id, dataset_id) pairs


In [12]:
# Extra pairs (False Positives)
extras = extracted_pairs - label_pairs
print(f"Extracted {len(extras)} pairs that were not in ground truth")

Extracted 19982 pairs that were not in ground truth


In [13]:
missed_df = pd.DataFrame(list(missed), columns=['article_id', 'dataset_id'])
missed_df.sort_values(by='article_id', inplace=True)
missed_df.head(25)

,article_id,dataset_id
100,10.1002_ecs2.4619,https://doi.org/10.25349/d9qw5x
23,10.1002_nafm.10870,https://doi.org/10.5066/p9gtumay
85,10.1007_s00259-022-06053-8,https://doi.org/10.7937/tcia.2019.30ilqfcl
58,10.1007_s00259-022-06053-8,https://doi.org/10.7937/k9/tcia.2017.7hs46erv
81,10.1007_s12559-020-09751-3,https://doi.org/10.7937/k9/tcia.2015.u1x8a5nr
74,10.1016_j.ast.2022.107401,https://doi.org/10.17862/cranfield.rd.19146182
89,10.1016_j.ast.2022.107401,https://doi.org/10.17862/cranfield.rd.19146182.v1
92,10.1016_j.cpc.2024.109087,https://doi.org/10.17632/9gr6pxhfjm
42,10.1016_j.fuel.2022.125768,https://doi.org/10.11583/dtu.20555586.v3
10,10.1016_j.fuel.2022.125768,https://doi.org/10.11583/dtu.20555586


In [14]:
missed_df.shape

(111, 2)

In [15]:
labels_df[labels_df['dataset_id'].str.contains("https://doi.org/")].shape

(325, 3)

In [16]:
missed_df[~missed_df['dataset_id'].str.contains("https://doi.org/")]

,article_id,dataset_id
